In [ ]:
import tensorflow as tf
from keras import layers
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
y_train = train_df['label']
X_train = train_df.drop('label', axis=1)
x_train = X_train/255
test=test_df/255

In [ ]:
x_train_array = np.array(x_train)
test_array = np.array(test)
x_train_array = x_train_array.reshape(42000, 28, 28, 1)
test_array = test_array.reshape(-1, 28, 28, 1)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="constant",
    zoom_range = 0.3,
    horizontal_flip=False,
    vertical_flip=False,
    
    rotation_range=30,
    
    validation_split=0.2
)

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

cnn1 = Sequential()
cnn1.add(Conv2D(48, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn1.add(BatchNormalization())

cnn1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.25))

cnn1.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(Dropout(0.25))

cnn1.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.25))
cnn1.add(Flatten())

cnn1.add(Dense(512, activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(Dropout(0.5))

cnn1.add(Dense(256, activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(Dropout(0.5))

cnn1.add(Dense(128, activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(Dropout(0.5))

cnn1.add(Dense(64, activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(Dropout(0.4))


cnn1.add(Dense(32, activation='relu'))
cnn1.add(BatchNormalization())
cnn1.add(Dropout(0.1))

cnn1.add(Dense(10, activation='softmax'))

early_stopper = EarlyStopping(monitor='loss', patience=15, min_delta = 1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)

In [ ]:
cnn1.summary()

In [ ]:
cnn1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = cnn1.fit(x_train_array,y_train,
          epochs=100,
          callbacks=[early_stopper, reduce_lr],
          validation_split=0.1
          )

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_test1 = cnn1.predict(test_array)
y_test1 = np.array(pd.DataFrame(y_test1).idxmax(axis=1))

In [ ]:
df_submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
df_submission['Label']=y_test1
df_submission.to_csv('submission.csv',index=False)